# Genetic Algorithm Example

## 1. Problem Definition

Let consider an equation:

$$Y = 15 \times x - x^2$$

Find $x \in [0, 15] \in \mathbb{R}$ such that it maximizes the output $Y$. The problem also is expressed as:

$$P = \max_{x}{Y}$$

## 2. Implementation

In this example, we will try to implement a Binary-coded Genetic Algorithm from scratch to solve a simple optimization problem.



We first import the necessary libraries

In [1]:
import numpy as np
from numpy import random as rd
import matplotlib as plt
np.random.seed(0)

First, we define the `convert_list_to_bitstring()` and `decode_string()` functions to convert a binary list to bitstring and decode the bitstring to corresponding positive integer

In [2]:
def convert_list_bitstring(x: list):
    s = ''.join(str(b) for b in x)
    return s

In [3]:
def decode_string(x: str):
    d = int(x, 2)
    return d

Given we have implemented the Genetic Algorithm to maximize fitness function $Y$. So we define `fitness()` function that take a bitstring as input, decode it back to integer and return the corresponding value following the $Y$ formula.

In [4]:
def fitness(x: str):
    # decode bitstring back to integer
    x_decoded = decode_string(x)
    # compute the fitness function
    y = 15 * x_decoded - x_decoded ** 2
    return y

Now, we implement the `roulette_selection()` function that using Roulette wheel selection algorithm. The function will take the population `pop`, corresponding fitness values `scores` and the size of matting pool `n` as inputs and then return the matting pool that contains individuals used to reproduction. In this case, the size of matting pool with be equal to the population size.

In [5]:
def roulette_selection(pop: list, scores: str, n: int):
    # compute the sum of fitness
    fitness_total = float(sum(scores))
    # compute probability of fitness
    fitness_prob = [f / fitness_total for f in scores]
    # compute cumulative probability of fitness
    fitness_cum_prob = [sum(fitness_prob[: i + 1]) for i in range(len(fitness_prob))]
    # list of selected parents
    pop_selected = []
    for _ in range(n):
        r = rd.uniform()
        for i, p in enumerate(pop):
            if r <= fitness_cum_prob[i]:
                pop_selected.append(p)
                break
    return pop_selected

Now, we implement one point crossover. This `crossover()` function will take two parents and the crossover rate. The crossover rate is a hyperparameter that determines whether crossover is performed or not, and if not, the parents are copied into the next generation.

In [6]:
def crossover(p1: list, p2: list, pc: float):
    # children are copies of parents by default
    c1, c2 = p1, p2
    # check for recombination
    if rd.rand() < pc:
        # select crossover point that is not on the end of the string
        pt = rd.randint(1, len(p1)-2)
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return c1, c2

The `mutation()` function flips bits with low probability `pm`.

In [7]:
def mutation(x: str, pm: float):
    for i in range(len(x)):
        # check for a mutation
        if rd.rand() < pm:
            # flip the bit
            x = x[: i] + str(1 - int(x[i])) + x[i + 1:]

After that, we can tie all of this together into a function named `genetic_algorithm()` that takes the name of the objective function and the hyperparameters of the search, and returns the best solution found during the search.

In [8]:
def genetic_algorithm(fitness_func: callable, n_bits: int, n_iters: int, n_pop: int, pc: float, pm: float):
    # initial population of random bitstring
    pop = [convert_list_bitstring(rd.randint(0, 2, n_bits).tolist()) for _ in range(n_pop)]
    # keep track of best solution
    best, best_eval = pop[0], fitness_func(pop[0])
    # enumerate generations
    for gen in range(n_iters):
        # evaluate all candidates in the population
        scores = [fitness_func(c) for c in pop]
        # check for new best solution
        for i in range(n_pop):
            if scores[i] > best_eval:
                best, best_eval = pop[i], scores[i]
                print(">%d, new best f(%s) = %.3f" % (gen,  pop[i], scores[i]))
        # select parents
        selected = roulette_selection(pop, scores, n_pop)
        # create the next generation
        children = []
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i + 1]
            # crossover and mutation
            for c in crossover(p1, p2, pc):
                # mutation
                mutation(c, pm)
                # store for next generation
                children.append(c)
        # replace old population with new one that having high scores
        pop = children
    return best, best_eval

Now, we create an initial population using binary representation. Due to $x \in [0, 15] \in \mathbb{R}$, the length of bit string is chosen as $4$. Therefore, our population is basically a set of random bit string. For the ease of understanding we are taking $5$ generations and $6$ solutions per population. We will use a crossover rate of $0.7$ and a mutation rate of $0.001$.

In [9]:
# define the length of bit string
n_bits = 4
# define number of generations for finding the solution
n_iters = 5
# define the polulation size
n_pop = 6
# define crossover rate
pc = 0.7
# define mutation rate
pm = 0.001

In [10]:
# perform the genetic algorithm search
best, score = genetic_algorithm(fitness, n_bits, n_iters, n_pop, pc, pm)
print('Done!')
print(f'bistring: {best}, decoded integer: {decode_string(best)}, f({decode_string(best)}) = {score}')

Done!
bistring: 0110, decoded integer: 6, f(6) = 54


Then we try with $50$ iterations and $8$ solutions per population with mutation rate of $0.01$ and crossover rate of $0.8$.

In [11]:
# define number of generations for finding the solution
n_iters = 50
# define the polulation size
n_pop = 8
# define crossover rate
pc = 0.8
# define mutation rate
pm = 0.01
# perform the genetic algorithm search
best, score = genetic_algorithm(fitness, n_bits, n_iters, n_pop, pc, pm)
print('Done!')
print(f'bistring: {best}, decoded integer: {decode_string(best)}, f({decode_string(best)}) = {score}')

>0, new best f(1101) = 26.000
>0, new best f(0100) = 44.000
>0, new best f(1010) = 50.000
>0, new best f(0111) = 56.000
Done!
bistring: 0111, decoded integer: 7, f(7) = 56
